In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
#init_notebook_mode(connected=True)  
#import cufflinks as cf  
#cf.go_offline() 

def logistic(x):   
    if x >= 0:
        z = np.exp(-x)
        a=1 / (1 + z)
    else:        
        z = np.exp(x)
        a= z / (1 + z)
    return  a

def logistic_deriv(x):
    return logistic(x) * (1 - logistic(x))

LR = 1   

I_dim = 17
H_dim = 3

epoch_count = 1

In [282]:
training_data = pd.read_csv('Absenteeism_at_work.csv', sep = ';')
training_data.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,1
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,1
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,1
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,1


In [283]:
correct_classification_count = 0

In [284]:
training_data['Absenteeism time in hours'].replace(0,"NOT ABSENT",inplace=True)
training_data['Absenteeism time in hours'].replace([1,2,3,4,5,6,7,8,9,10,16],"DAYS",inplace=True)
training_data['Absenteeism time in hours'].replace([16,24,40,32,56,48],"WEEK",inplace=True)
training_data['Absenteeism time in hours'].replace([120,80,64,112,104],"MONTH",inplace=True)
training_data['Absenteeism time in hours'].value_counts()

DAYS          696
NOT ABSENT     44
Name: Absenteeism time in hours, dtype: int64

In [285]:
training_data.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,DAYS
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,NOT ABSENT
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,DAYS
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,DAYS
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,DAYS


In [286]:
training_data = pd.read_csv('Absenteeism_at_work.csv', sep = ';')
training_data.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,1
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,1
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,1
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,1


In [287]:
training_data['Absenteeism time in hours'].value_counts()

1    696
0     44
Name: Absenteeism time in hours, dtype: int64

In [288]:
#np.random.seed(1)
weights_ItoH = np.random.uniform(-1, 1, (I_dim, H_dim))
weights_HtoO = np.random.uniform(-1, 1, H_dim)

preActivation_H = np.zeros(H_dim)
postActivation_H = np.zeros(H_dim)

training_data = pd.read_excel("Absenteeism_at_work.xls")
training_data.rename(columns={"Absenteeism time in hours": "Default"}, inplace=True)
training_data.drop('ID', axis = 1, inplace =True) # drop column "ID"
#training_data['Defaul'].unique()



target_output = training_data['Default']
training_data = training_data.drop('Default',axis = 1)

training_data = training_data.drop('Month of absence',axis = 1)
training_data = training_data.drop('Service time',axis = 1)

training_data = np.asarray(training_data)
training_count = len(training_data[:,0])

In [289]:
validation_data = pd.read_excel("Absenteeism_at_work.xls")
validation_data.rename(columns={"Absenteeism time in hours": "Default"}, inplace=True)
validation_data.drop('ID', axis = 1, inplace =True) # drop column "ID"
validation_data.drop('Month of absence', axis = 1, inplace =True) # drop column "ID"
validation_data.drop('Service time', axis = 1, inplace =True) # drop column "ID"
validation_data['Default'].unique()
validation_output = validation_data['Default']
validation_data = validation_data.drop('Default', axis=1) 
validation_data = np.asarray(validation_data)
validation_count = len(validation_data[:,0])

In [290]:

#------------------------------training----------------------------

for epoch in range(epoch_count):
    for sample in range(training_count):
        for node in range(H_dim):
            preActivation_H[node] = np.dot(training_data[sample,:], weights_ItoH[:, node])
            postActivation_H[node] = logistic(preActivation_H[node])
            
        preActivation_O = np.dot(postActivation_H, weights_HtoO)
        postActivation_O = logistic(preActivation_O)
        
        FE = postActivation_O - target_output[sample]
        
        for H_node in range(H_dim):
            S_error = FE * logistic_deriv(preActivation_O)
            gradient_HtoO = S_error * postActivation_H[H_node]
                       
            for I_node in range(I_dim):
                input_value = training_data[sample, I_node]
                gradient_ItoH = S_error * weights_HtoO[H_node] * logistic_deriv(preActivation_H[H_node]) * input_value
                
                weights_ItoH[I_node, H_node] -= LR * gradient_ItoH
                
            weights_HtoO[H_node] -= LR * gradient_HtoO


In [291]:

#------------------------------------validation--------------------------
           

for sample in range(validation_count):
    for node in range(H_dim):
        preActivation_H[node] = np.dot(validation_data[sample,:], weights_ItoH[:, node])
        postActivation_H[node] = logistic(preActivation_H[node])
            
    preActivation_O = np.dot(postActivation_H, weights_HtoO)
    postActivation_O = logistic(preActivation_O)
        
    if postActivation_O > 0.5:
        output = 1
    else:
        output = 0     
        
    if output == validation_output[sample]:
        correct_classification_count += 1

print('Percentage of correct classifications:')
print(correct_classification_count*100/validation_count)

Percentage of correct classifications:
5.945945945945946
